In [1]:
import os,sys
import mlflow
%pwd

'd:\\collab_projects\\Crop_prediction\\notebooks'

In [2]:
os.chdir("../")
%pwd

'd:\\collab_projects\\Crop_prediction'

In [3]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/gbiamgaurav/Crop_prediction.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "gbiamgaurav"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "352861024ebede76aa75cf4ea021ee1f74b0e39f"


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from cropPrediction.constants import * 
from cropPrediction.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/gbiamgaurav/Crop_prediction.mlflow",
           
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score
from urllib.parse import urlparse
import numpy as np
import mlflow
import mlflow.sklearn
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        acc = accuracy_score(actual, pred)
        p_score = precision_score(actual, pred, average="weighted")
        return acc, p_score
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predictions = model.predict(test_x)

            (acc, p_score) = self.eval_metrics(test_y, predictions)
            
            # Saving metrics as local
            scores = {"Accuracy Score": acc, "Precision Score": p_score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("Accuracy Score", acc)
            mlflow.log_metric("Precision Score", p_score)
            


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (acc, p_score) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"Accuracy Score": acc, "Precision Score": p_score}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2023-09-28 14:31:53,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-28 14:31:53,645: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 14:31:53,648: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 14:31:53,648: INFO: common: created directory at: artifacts]
[2023-09-28 14:31:53,653: INFO: common: created directory at: artifacts/model_evaluation]
[2023-09-28 14:31:55,862: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Successfully registered model 'RandomForestModel'.
2023/09/28 14:32:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestModel, version 1
Created version '1' of model 'RandomForestModel'.


[2023-09-28 14:32:37,446: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
